In [3]:
import torch
from torch.nn import Module
import torch.nn.functional as F
import brevitas.nn as qnn
from brevitas.core.quant import QuantType
from CG4002.MachineLearning.main_code.util_main import load_dance_dance_data_set
from CG4002.MachineLearning.main_code.util_main import test_model_nn
from CG4002.MachineLearning.main_code.util_main import load_model as load_model_keras
from CG4002.MachineLearning.main_code.util_main import one_hot_encode_labels
import torch 
import numpy as np
import torch.nn as nn
from sklearn.utils import shuffle

data_set_path = "CG4002/MachineLearning/main_code/Data-sets/Dance_Data"
sampling_rate = 5
window_length = 2.56  # Seconds

#load data set
x_data, y_data = load_dance_dance_data_set(data_set_path, sampling_rate=sampling_rate, window_length=window_length)

x_data, y_data = shuffle(x_data, y_data)
y_data = y_data - 1

x_train_data, x_test_data = x_data[:int(len(x_data)*0.7), :], x_data[int(len(x_data)*0.7):, :]
y_train_data, y_test_data = y_data[:int(len(x_data)*0.7), :], y_data[int(len(x_data)*0.7):, :]
#y_data = one_hot_encode_labels(y_data)

x_train = torch.from_numpy(x_train_data).float()
y_train = torch.from_numpy(y_train_data).float()
y_train = y_train.reshape(y_train.shape[0]).long()

x_test = torch.from_numpy(x_test_data).float()
y_test = torch.from_numpy(y_test_data).float()
y_test = y_test.reshape(y_test.shape[0]).long()

input_dim = x_data.shape[1]
output_dim = one_hot_encode_labels(y_data).shape[1]

Processing: dumbbells.txt
(299, 72)
Processing: face_wipe.txt
(299, 72)
Processing: left.txt
(299, 72)
Processing: muscle.txt
(299, 72)
Processing: nothing.txt
(299, 72)
Processing: pac_man.txt
(299, 72)
Processing: right.txt
(299, 72)
Processing: shooting_star.txt
(299, 72)
Processing: shout_out.txt
(299, 72)
Processing: tornado.txt
(299, 72)
Processing: weight_lifting.txt
(299, 72)


In [4]:
np.save('inputs.npy', x_data, allow_pickle=False, fix_imports=False)
np.save('labels.npy', y_data, allow_pickle=False, fix_imports=False)

In [9]:
#hardtanh version
class MLP_quantized_exportable(Module):
    def __init__(self):
        super(MLP_quantized_exportable, self).__init__()
        self.input   = qnn.QuantLinear(input_dim, 256, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.input_relu = qnn.QuantHardTanh(quant_type=QuantType.INT, bit_width=4)
        
        self.h1   = qnn.QuantLinear(256, 256, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.relu1 = qnn.QuantHardTanh(quant_type=QuantType.INT, bit_width=4)
        
        self.h2   = qnn.QuantLinear(256, 128, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.relu2 = qnn.QuantHardTanh(quant_type=QuantType.INT, bit_width=4)
        
        self.h3   = qnn.QuantLinear(128, output_dim, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=4)

        self.output   = qnn.QuantSigmoid(quant_type=QuantType.FP, bit_width=8)

    def forward(self, x):
        x = self.input(x)
        x = self.input_relu(x)
        
        x = self.h1(x)
        x = self.relu1(x)
        
        x = self.h2(x)
        x = self.relu2(x)
        
        x = self.h3(x)
        return self.output(x)
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [18]:
from torch.utils import data
model = MLP_quantized_exportable()

class Dataset_custom(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, inputs, labels):
        'Initialization'
        self.labels = labels
        self.inputs = inputs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.inputs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        x = self.inputs[index]
        y = self.labels[index]

        return x, y
    
train_dataset = Dataset_custom(x_train, y_train) # create your dataset
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = Dataset_custom(x_test, y_test) # create your dataset
test_loader = data.DataLoader(test_dataset, batch_size=32)




In [20]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



def score_function(engine):
    val_loss = engine.state.metrics['crossentropy']
    accuracy = engine.state.metrics['accuracy']
    return accuracy #-val_loss



trainer = create_supervised_trainer(model, optimizer, loss)
evaluator = create_supervised_evaluator(model,
                                        metrics={
                                            'accuracy': Accuracy(),
                                            'crossentropy': Loss(loss)
                                            })

handler = EarlyStopping(patience=5, score_function=score_function, trainer=trainer)
# Note: the handler is attached to an *Evaluator* (runs one epoch on validation dataset).
evaluator.add_event_handler(Events.COMPLETED, handler)

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['crossentropy']))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(test_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['crossentropy']))

trainer.run(train_loader, max_epochs=35)

torch.save(model.state_dict(), "quantized_statedict_exportable.pt")

Training Results - Epoch: 1  Avg accuracy: 0.91 Avg loss: 1.57
Validation Results - Epoch: 1  Avg accuracy: 0.89 Avg loss: 1.57
Training Results - Epoch: 2  Avg accuracy: 0.91 Avg loss: 1.56
Validation Results - Epoch: 2  Avg accuracy: 0.89 Avg loss: 1.57
Training Results - Epoch: 3  Avg accuracy: 0.94 Avg loss: 1.57
Validation Results - Epoch: 3  Avg accuracy: 0.94 Avg loss: 1.58
Training Results - Epoch: 4  Avg accuracy: 0.92 Avg loss: 1.57
Validation Results - Epoch: 4  Avg accuracy: 0.91 Avg loss: 1.58
Training Results - Epoch: 5  Avg accuracy: 0.93 Avg loss: 1.56
Validation Results - Epoch: 5  Avg accuracy: 0.92 Avg loss: 1.58
